In [1]:
import os
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import operator
import json
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn import svm
from collections import Counter
from sklearn.preprocessing import LabelEncoder, scale, PolynomialFeatures
from sklearn.datasets import load_boston
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from datetime import datetime
from scipy import stats
import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from torch.autograd import Variable

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
train_csv = pd.read_csv("Dataset/train.csv", keep_default_na=False)
test_csv = pd.read_csv("Dataset/test.csv", keep_default_na=False)

data = {}
labels = {}

In [3]:
train_csv.shape

(5959, 3)

In [4]:
def preprocess_data(dataset):
    dataset['Review Text'] = dataset["Review Title"].map(str) + " " + dataset['Review Text']    
    dataset = dataset.drop(['Review Title'],axis=1)
    return dataset

In [5]:
train = preprocess_data(train_csv)
test = preprocess_data(test_csv)

train_X = train["Review Text"]
train_y = train["topic"]
test_X = test["Review Text"]

#train_X, test_X, train_y, test_y = train_test_split(train["Review Text"], train["topic"], random_state=33)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))
#s = " ".join(train["Review Text"][0:10])
#print(s)

IMDb reviews (combined): train = 5959, test = 2553


In [6]:
import numpy as np
import re
from collections import Counter

def build_dict(data, vocab_size = 6300):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    word_counts = Counter(np.concatenate( data, axis=0 ))
    # word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    print(len(sorted_words))

    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [7]:
#word_dict = build_dict(train_X)
# print(word_dict)
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    #words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [8]:
print(train_X[0])
review_to_words(train_X[0])

Useless Did nothing for me, didn't help lost even with working out and eating healthy. Didn't curb appetite or anything.


['useless',
 'nothing',
 'help',
 'lost',
 'even',
 'working',
 'eating',
 'healthy',
 'curb',
 'appetite',
 'anything']

In [9]:
import pickle

cache_dir = os.path.join("cache", "key_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    print(cache_file)
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            print("File not found")
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        print('Preprocess training and test data to obtain words for each review')
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        print('Preprocess training and test data to obtain words for each review')
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        print('Unpack data loaded from cache file')
        # Unpack data loaded from cache file
        words_train, words_test, labels_train = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'])
    
    return words_train, words_test, labels_train

In [10]:
# Preprocess data
train_X, test_X, train_y = preprocess_data(train_X, test_X, train_y)

preprocessed_data.pkl
File not found
Preprocess training and test data to obtain words for each review
Preprocess training and test data to obtain words for each review
Wrote preprocessed data to cache file: preprocessed_data.pkl


In [11]:
word_dict = build_dict(train_X)

8757


In [12]:
# determine the five most frequently appearing words in the training set.
word_counts = Counter(np.concatenate( train_X, axis=0 ))
sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
print(sorted_words[0:5])
print(word_counts['tast'])

['taste', 'product', 'like', 'bad', 'brand']
2


In [13]:
#word_dict

In [14]:
with open('word_dict.json', 'w') as f:
    json.dump(word_dict, f)

In [15]:
with open(os.path.join(cache_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [16]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [17]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [18]:
print(len(train_X[0]))
print(train_X_len[0:1]) # 500 - 322 (non zeros) = 178

500
[11]


In [19]:
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1).to_csv('sample_train.csv', header=False, index=False)
pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1).to_csv('sample_test.csv', header=False, index=False)

In [20]:
class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.output_size = 1
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=21)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.08
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.lstm.weight_ih_l0.data.uniform_(-initrange, initrange)
        self.lstm.weight_hh_l0.data.uniform_(-initrange, initrange)
        
        self.lstm.bias_ih_l0.data.zero_()
        self.lstm.bias_hh_l0.data.zero_()
        
        # self.fc.bias.data.zero_()
        self.dense.bias.data.fill_(0)
        # self.fc.weight.data.uniform_(-initrange, initrange)
        self.dense.weight.data.normal_(0.0, (1.0 / np.sqrt(self.dense.in_features)))
        
    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        
        batch_size = x.size(0)
        
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        
        #print(reviews)
        #print(batch_size)
        
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dense(lstm_out)
        #print(out)
        #sig_out = out.view(batch_size, -1, self.output_size)
        #sig_out = sig_out[:, -1]
        #print(sig_out)
        
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [21]:
import torch.nn as nn

class DNNClassifier(nn.Module):
    """
    This is the simple DNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, hidden_dim, input_size, output_size, dropout=0.5):
        """
        Initialize the model by settingg up the various layers.
        """
        super(DNNClassifier, self).__init__()

        self.sig = nn.Sigmoid()        
        # self.word_dict = None
        
        self.fc1 = nn.Linear(input_size, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)
        self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(p=0.5)
        self.init_weights()
        self.soft = nn.LogSoftmax(dim=0)
        
    def init_weights(m):
        initrange = 0.08
        classname = m.__class__.__name__
        if classname.find('Linear') != -1:
            # get the number of the inputs
            n = m.in_features
            y = 1.0/np.sqrt(n)
            m.weight.data.normal_(0.0, y)
            m.bias.data.fill_(0)
        
    def forward(self, input_x):
        """
        Perform a forward pass of our model on some input.
        """
        #input_x = input_x.t()
        #lengths = input_x[0,:]        
        x = input_x[0:,1:]
        #print(x)
        #print(input_size)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        out = self.out(x)
        return self.sig(out)

In [22]:
# Read in only the first 250 rows
train_sample = pd.read_csv('sample_train.csv', header=None, names=None)

binary_label = pd.get_dummies(train_sample[[0]].astype('category'), prefix="", prefix_sep="")

#binary_label_columns = binary_label.dot(binary_label.columns)
#binary_label_columns = binary_label.idxmax(axis=1)

#binary_label_stack = binary_label.stack()
#print(binary_label.columns[0:20])
#print(binary_label_columns)
#print(binary_label_columns[5].replace("0_", ""))

labels = np.asarray(binary_label)

lbl = LabelEncoder() 
lbl.fit(list(train_sample[[0]].values)) 
print(lbl.transform(train_sample[[0]].values))
# Turn the input pandas dataframe into tensors

#train_sample_y = torch.from_numpy(lbl.transform(list(train_sample[[0]].values))).float().squeeze()
train_sample_y = labels
train_sample_X = train_sample.drop([0], axis=1).values

#train_sample_y = torch.from_numpy(labels).float().squeeze()
#train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

print(train_sample_y)

[15 11  4 ...  1  1  1]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [23]:
# Hyperparameters
batch_size = 50
num_epochs = 30
# Learning Rate
learning_rate = 0.0001
# Model parameters
# Input size
input_size = train_sample_X.shape[1] - 1
# Output size
output_size = 21
# Embedding Dimension
embedding_dim = 32
# Hidden Dimension
hidden_dim = 128
# Number of RNN Layers
n_layers = 2
#vocabulary size
vocab_size = 6300

# Show stats for every n number of batches
show_every_n_batches = 50

In [24]:
X_train, X_val, y_train, y_val = train_test_split(train_sample_X, train_sample_y, random_state=33)

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(torch.from_numpy(X_train).long(), torch.from_numpy(y_train).long().squeeze())
val_sample_ds = torch.utils.data.TensorDataset(torch.from_numpy(X_val).long(), torch.from_numpy(y_val).long().squeeze())
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=batch_size)
var_sample_dl = torch.utils.data.DataLoader(val_sample_ds, batch_size=batch_size)

In [25]:
def train(model, train_loader, val_loader, epochs, optimizer, loss_fn, device):
    train_loss = 0
    val_loss = 0
    for epoch in range(1, epochs + 1):
        model.train()
        for batch in train_loader:
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            model.zero_grad()
            optimizer.zero_grad()
            
            #batch_len = batch_X.t()[0,:]
            #print(batch_len)
            
            # TODO: Complete this train method to train the model provided.
            output = model.forward(batch_X)
            #print(output.reshape(-1,1).shape)
            #print(batch_y)
            _, batch_y = batch_y.max(dim=1)            
            batch_y = torch.autograd.Variable(batch_y)
            #batch_y = batch_y.reshape(-1,1)
            #print(output)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            #print(loss.item())
            #break
        model.eval()
        for batch in val_loader:
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # model.zero_grad()
            #optimizer.zero_grad()
            # TODO: Complete this train method to train the model provided.
            output = model.forward(batch_X)
            
            _, batch_y = batch_y.max(dim=1)            
            batch_y = torch.autograd.Variable(batch_y)
            #batch_y = batch_y.reshape(-1,1)
            #print(output)
            
            loss = loss_fn(output, batch_y)            
            val_loss += loss.item()
        
        total_train_loss = train_loss / len(train_loader)
        total_val_loss = val_loss / len(val_loader)
        
        print("Epoch: {}, BCE Train Loss: {} Valid Loss {}".format(epoch, total_train_loss, total_val_loss))
        val_loss = 0
        train_loss = 0
        #break
    return model

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [27]:
#model = DNNClassifier(hidden_dim, input_size, output_size).to(device)
model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.NLLLoss()
#loss_fn = torch.nn.BCELoss()
#loss_fn = nn.MultiLabelSoftMarginLoss()
#loss_fn = nn.BCEWithLogitsLoss()

model_tn = train(model, train_sample_dl, var_sample_dl, num_epochs, optimizer, loss_fn, device)

Epoch: 1, BCE Train Loss: 3.0394735866122775 Valid Loss 3.0259478171666463
Epoch: 2, BCE Train Loss: 2.8694888406329686 Valid Loss 2.806047487258911
Epoch: 3, BCE Train Loss: 2.793511594666375 Valid Loss 2.7886878490447997
Epoch: 4, BCE Train Loss: 2.781208703253004 Valid Loss 2.7811298688252766
Epoch: 5, BCE Train Loss: 2.7751123825709025 Valid Loss 2.776583536465963
Epoch: 6, BCE Train Loss: 2.77117244137658 Valid Loss 2.77344434261322
Epoch: 7, BCE Train Loss: 2.7682804743448894 Valid Loss 2.7711456457773846
Epoch: 8, BCE Train Loss: 2.7660461717181737 Valid Loss 2.769388516743978
Epoch: 9, BCE Train Loss: 2.764266077677409 Valid Loss 2.7679794311523436
Epoch: 10, BCE Train Loss: 2.7627943939632837 Valid Loss 2.7667961915334067
Epoch: 11, BCE Train Loss: 2.761523699760437 Valid Loss 2.765746800104777
Epoch: 12, BCE Train Loss: 2.760365194744534 Valid Loss 2.764759580294291
Epoch: 13, BCE Train Loss: 2.75918726656172 Valid Loss 2.7638048410415648
Epoch: 14, BCE Train Loss: 2.75777210

In [28]:
with open('save/trained_rnn_new', 'wb') as pickle_file:
    torch.save(model_tn, pickle_file)

In [29]:
# Read in only the first 250 rows
test_sample_X = pd.read_csv('sample_test.csv', header=None, names=None)

#test_sample_ds = torch.utils.data.TensorDataset(torch.from_numpy(X_train).long(), torch.from_numpy(y_train).float().squeeze())
#test_sample_dl = torch.utils.data.DataLoader(test_sample_ds, batch_size=batch_size)
#print(train_sample_y)

In [30]:
model_tn = torch.load("save/trained_rnn_new")
model_tn.eval()

LSTMClassifier(
  (embedding): Embedding(6300, 32, padding_idx=0)
  (lstm): LSTM(32, 128)
  (dense): Linear(in_features=128, out_features=21, bias=True)
  (sig): Sigmoid()
)

In [31]:
precdictions = []
for test_x in train_sample_X:
    test_x = test_x.reshape(1, -1)
    test_x = torch.from_numpy(test_x).long()
    test_x = test_x.to(device)
    test_x = torch.autograd.Variable(test_x)
    #test_x = torch.from_numpy(test_x).long().unsqueeze(0)
    #test_x = test_x.to(device)
    #print(test_x)
    with torch.no_grad():
        output = model_tn.forward(test_x)
    #print(output.mean())
    #result = 1/np.log(output.numpy())
    output = output.squeeze().cpu().detach().numpy()
    #output = output.detach().numpy()
    #print(output)
    #probs = np.log(1 - torch.sigmoid(output))
    #print(probs)
    precdictions.append(np.argmax(output) + 1)

print(precdictions)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 21, 2, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 16, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 21, 21, 2, 2, 2, 2, 2,

In [32]:
input_data = "These are GAWD awful, they do not stick, they are uncomfortable because it doesn't stick so it moves around making it very uncomfortable and difficult to absorb. These are HORRIBLE"
data_X = None
data_len = None
input_data_words = review_to_words(input_data)
data_X, data_len = convert_and_pad(word_dict, input_data_words)
# data_X = pd.concat([pd.DataFrame(test_data_len), pd.DataFrame(test_data)], axis=1)

# Using data_X and data_len we construct an appropriate input tensor. Remember
# that our model expects input data of the form 'len, review[500]'.
# data_pack = np.hstack((data_len, data_X))
data_pack = np.hstack((data_len, data_X))

data_pack = data_pack.reshape(1, -1)

data = torch.from_numpy(data_pack).long()
data = data.to(device)

# Make sure to put the model into evaluation mode
model_tn.eval()

# TODO: Compute the result of applying the model to the input data. The variable `result` should
#       be a numpy array which contains a single integer which is either 1 or 0
#print(data)
with torch.no_grad():
    output = model_tn.forward(data)

#print(output)
probs = np.log(1 - torch.sigmoid(output))
#probs = torch.sigmoid(output)
#print(probs)
#print(np.argmax(probs))
#predicted_vals = probs > threshold

#print(predicted_vals)
#result = np.round(output.numpy())
# result = predictor.predict(data.values)
#print(result)

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
train_sample["topic_test"] = pd.Series(lbl.inverse_transform(np.array(precdictions)))

In [ ]:
train_sample.head()

In [ ]:
train_sample.to_csv("submission.csv", index=False)

In [ ]:
# Fake NN output
out = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9], [0.9, 0.05, 0.05]])
out = torch.autograd.Variable(out)

# Categorical targets
y = torch.LongTensor([1, 2, 0])
y = torch.autograd.Variable(y)

# One-hot encoded targets
y1 = torch.FloatTensor([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
y1 = torch.autograd.Variable(y1)

print(out)
print(y1)

# Calculating the loss
loss_val = nn.CrossEntropyLoss()(out, y)
loss_val1 = nn.BCEWithLogitsLoss()(out, y1)

print(loss_val)
print(loss_val1)